In [1]:
import pandas as pd
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import os
import urllib.parse


# 공통 파라미터 설정
base_url = "https://search.naver.com/search.naver"
common_params = {
    'where': 'news',
    'query': '삼성전자',
    'sm': 'tab_opt',
    'sort': '1',
    'photo': '0',
    'field': '0',
    # 'pd': '3',
    'pd': '7',
    # 'ds': '2024.04.23',
    # 'de': '2024.04.23',
    'docid': '',
    'related': '0',
    'mynews': '1',
    'office_type': '0',
    'office_section_code': '0',
    'news_office_checked': '',
    # 'nso': 'so:dd,p:from20240423to20240423',
    'is_sug_officeid': '0',
    'office_category': '0',
    'service_area': '0',
}

encoded_params = urllib.parse.urlencode(common_params)
full_url = f"{base_url}?{encoded_params}"

print("URL:", full_url)


# Initialize a Chrome webdriver
op = webdriver.ChromeOptions()
op.add_argument('--headless')
op.add_argument('--no-sandbox')
op.add_argument("--disable-dev-shm-usage")
# op.add_argument("--window-size=1920x1080")

# service.start()
# driver = webdriver.Remote(service.service_url, options=op)
# driver = webdriver.Chrome(service=Service(ChromeDriverManager(version="119.0.6045.105").install()), options=op)
service = Service("./chromedriver-linux64/chromedriver")
driver = webdriver.Chrome(service=service, options=op)
SCROLL_PAUSE_TIME = 0.5
driver.get(full_url)

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height


# Wait for the dynamic content to load
time.sleep(1)

# Parse the webpage with BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

URL: https://search.naver.com/search.naver?where=news&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&sm=tab_opt&sort=1&photo=0&field=0&pd=7&docid=&related=0&mynews=1&office_type=0&office_section_code=0&news_office_checked=&is_sug_officeid=0&office_category=0&service_area=0


In [2]:
articles = soup.select("#main_pack > section.sc_new.sp_nnews._fe_news_collection._prs_nws > div.api_subject_bx > div.group_news > ul")
print(len(articles[0]))
news_links = []
for i in range(len(articles[0])):
    article = articles[0].select(f"#sp_nws{i+1} > div > div > div.news_info > div.info_group > a:nth-child(4)")
    if article == []:
        article = articles[0].select(f"#sp_nws{i+1} > div > div > div.news_info > div.info_group > a:nth-child(3)")
        if article == []:
            continue
    news_links.append(article[0]["href"])
driver.close()

78


In [3]:
news_links

['https://n.news.naver.com/mnews/article/029/0002871464?sid=101',
 'https://n.news.naver.com/mnews/article/079/0003891436?sid=101',
 'https://n.news.naver.com/mnews/article/011/0004335821?sid=105',
 'https://n.news.naver.com/mnews/article/277/0005413457?sid=101',
 'https://n.news.naver.com/mnews/article/092/0002330081?sid=105',
 'https://n.news.naver.com/mnews/article/092/0002330081?sid=105',
 'https://n.news.naver.com/mnews/article/138/0002172464?sid=105',
 'https://n.news.naver.com/mnews/article/015/0004979760?sid=105',
 'https://n.news.naver.com/mnews/article/014/0005179828?sid=101',
 'https://n.news.naver.com/mnews/article/014/0005179829?sid=101']

In [4]:
for link in news_links:
    driver.get(link)

InvalidSessionIdException: Message: invalid session id
Stacktrace:
#0 0x596b4f463e33 <unknown>
#1 0x596b4f1533b7 <unknown>
#2 0x596b4f19051d <unknown>
#3 0x596b4f1c0244 <unknown>
#4 0x596b4f1baee7 <unknown>
#5 0x596b4f1ba08f <unknown>
#6 0x596b4f11fb08 <unknown>
#7 0x596b4f42859b <unknown>
#8 0x596b4f42c34b <unknown>
#9 0x596b4f414681 <unknown>
#10 0x596b4f42ceb2 <unknown>
#11 0x596b4f3f96af <unknown>
#12 0x596b4f11e106 <unknown>
#13 0x737801029d90 <unknown>


In [1]:
driver.quit()

NameError: name 'driver' is not defined